# Time series predictor

In [1]:
import numpy as np
import time
from datetime import date
import datetime
from datetime import timedelta  
import csv
import holidays # for importing the public holidays
import re
import torch
from src.utils import *
from src.data_miner import DataMiner

num_features = 5
local_holidays = holidays.Italy(prov='BO') # Get the holidays in Bologna, Italy :)

/home/fedebotu/Documents/good-night-ml


In [2]:
# Variables
data_dir = "data"
dataset = "data/LastSeenDataset.csv"

- Feature extraction: we first extract the features given the time series data of Telegram accesses.
- Supposition: last Telegram access in very similar to the time the person goes to sleep

## Feature engineering
Possible features to extract: 
1. Last seen time (arguably the most important)
2. Wake up time
3. Number of Telegram accesses during the previous day
4. Day of the week
5. Public holiday presence in the following day (using the holidays library)
6. (time spent on Telegram)


In [3]:
with open(dataset, newline='') as csvfile:
    date_list = list(csv.reader(csvfile))

date_list = convert_to_dates(date_list)

'''Test data: search calendar for local holidays'''
print("First day is holiday: ", date_list[0][0] in local_holidays)

First day is holiday:  False


In [4]:
data_tensor =  DataMiner(date_list).to_tensor(verbose=False)
print(data_tensor)

tensor([[0.6002, 0.5434, 0.4465, 0.5033, 0.4888, 0.5380, 0.5200, 0.6680, 0.1981,
         0.5418, 0.5891, 0.5230, 0.5878, 0.2870, 0.1545, 0.3483, 0.1007, 0.6694,
         0.5091, 0.4906, 0.6093],
        [0.6667, 0.5991, 0.6653, 0.6445, 0.7801, 0.6894, 0.6742, 0.6647, 1.0048,
         0.6278, 0.7105, 0.6988, 0.6384, 0.8407, 0.9146, 0.7862, 1.1783, 0.4033,
         0.6723, 0.6988, 0.6034],
        [1.0000, 0.0000, 0.1667, 0.3333, 0.5000, 0.6667, 0.8333, 1.0000, 0.0000,
         0.1667, 0.3333, 0.5000, 0.6667, 0.8333, 1.0000, 0.0000, 0.1667, 0.3333,
         0.5000, 0.6667, 0.8333],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.1400, 0.1200, 0.0600, 0.0500, 0.0700, 0.1400, 0.0967, 0.1400, 0.0300,
         0.2100, 0.3400, 0.1400, 0.2600, 0.2200, 0.0500, 0.1700, 0.0300, 0.3900,
         0.3600, 0.2500, 0.2600]], dtype=torch.float64

# Data augmentation

Given that the training data is not much, we can insert some noise to augment it; this will also make the model less prone to overfitting

In [5]:
# Data augmentation


# We use the "last 3" trend
# Credits: https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/
'''The sequence on which we have a prediction is the last train_window days'''
train_window = 3
X, y = create_inout_sequences(data_tensor, train_window)
X = X.transpose(0, 2, 1)

## Model
- Time series data, so possible idea(s):
    - LSTM

In [6]:
import random
import numpy as np
import torch

# multivariate data preparation
from numpy import array
from numpy import hstack

# split a multivariate sequence into samples
# def split_sequences(sequences, n_steps):
#     X, y = list(), list()
#     for i in range(len(sequences)):
#         # find the end of this pattern
#         end_ix = i + n_steps
#         # check if we are beyond the dataset
#         if end_ix > len(sequences):
#             break
#         # gather input and output parts of the pattern
#         seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
#         X.append(seq_x)
#         y.append(seq_y)
#         print(X)
#     return array(X), array(y)

In [18]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 20 # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.rand(self.n_layers,batch_size,self.n_hidden)
        cell_state = torch.rand(self.n_layers,batch_size,self.n_hidden)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):        
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x,self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [19]:
n_features = num_features # this is number of parallel inputs
n_timesteps = train_window # this is number of timesteps

# convert dataset into input/output

# create NN
mv_net = MV_LSTM(n_features,n_timesteps)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-1)

train_episodes = 500
batch_size = 3
# n_features = 5 # this is number of parallel inputs
# n_timesteps = train_window # this is number of timesteps

# # convert dataset into input/output
# #X, y = split_sequences(data_tensor, n_timesteps)


# # create NN
# mv_net = MV_LSTM(n_features,n_timesteps)
# criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
# optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-1)

# train_episodes = 500
# batch_size = 1
print(X[2])
print(y[1])

[[0.44649306 0.66525465 0.16666667 0.         0.06      ]
 [0.50329864 0.64453703 0.33333334 0.         0.05      ]
 [0.48875    0.7801157  0.5        0.         0.07      ]]
[0.48875]


In [20]:
mv_net.train()
for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]
        x_batch = torch.tensor(inpt,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)
    
        mv_net.init_hidden(x_batch.size(0))
    #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
    #    lstm_out.contiguous().view(x_batch.size(0),-1)
        output = mv_net(x_batch) 
        loss = criterion(output, y_batch)  
#         print('PREDICTED:\n', output); print('REAL:\n', y_batch)
        loss.backward()
        optimizer.step()        
        optimizer.zero_grad()
    print('step : ' , t , 'loss : ' , loss.item())

# mv_net.train()
# for t in range(train_episodes):
#     for b in range(0,len(X),batch_size):
#         x_batch = X[b:b+batch_size]
#         y_batch = y[b:b+batch_size]    
#         mv_net.init_hidden(batch_size*torch.numel(X[b]))
#     #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
#     #    lstm_out.contiguous().view(x_batch.size(0),-1)
#         output = mv_net(x_batch)
#         loss = criterion(output.view(-1), y_batch)  
        
#         loss.backward()
#         optimizer.step()        
#         optimizer.zero_grad() 
#     print('step : ' , t , 'loss : ' , loss.item())

step :  0 loss :  0.03209098428487778
step :  1 loss :  0.05665023997426033
step :  2 loss :  0.033313628286123276
step :  3 loss :  0.005552265327423811
step :  4 loss :  0.003699325956404209
step :  5 loss :  0.012784675695002079
step :  6 loss :  0.01359355729073286
step :  7 loss :  0.0077173444442451
step :  8 loss :  0.005333678796887398
step :  9 loss :  0.006864545866847038
step :  10 loss :  0.007317788898944855
step :  11 loss :  0.005713809281587601
step :  12 loss :  0.007221674080938101
step :  13 loss :  0.009559947066009045
step :  14 loss :  0.009788010269403458
step :  15 loss :  0.007775808218866587
step :  16 loss :  0.009797773323953152
step :  17 loss :  0.008308322168886662
step :  18 loss :  0.009618685580790043
step :  19 loss :  0.0102749178186059
step :  20 loss :  0.009580097161233425
step :  21 loss :  0.011023461818695068
step :  22 loss :  0.010609623976051807
step :  23 loss :  0.011083393357694149
step :  24 loss :  0.009027763269841671
step :  25 loss :

In [33]:
with torch.no_grad():
    print('Predicted:', mv_net(torch.tensor(X[5:8,:,:],dtype=torch.float32))[0])
    print('Real:', y[5])

Predicted: tensor([0.2314])
Real: [0.19805556]


In [11]:
from torch import nn

'''We use a model which should predict time series data (e.g. RNN, LSTM, Transformer)'''
class LSTM(nn.Module):
    def __init__(self, input_size=num_features, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

LSTM(
  (lstm): LSTM(5, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [12]:
epochs = 150

for i in range(epochs):
    for seq, label in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        #print(seq)
        y_pred = model(seq)

        single_loss = loss_function(y_pred, label)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

NameError: name 'train_inout_seq' is not defined